# **Demo: Text Summarizer**

In this demo, you will create an Arxiv paper summarizer that downloads a paper as a PDF, reads it in one shot, and generates a summary. You can use the **The Impact of Generative Artificial Intelligence** paper as an example for demonstration.

## **Steps to Perform:**

*   Step 1: Import the Necessary Libraries
*   Step 2: Download and Read the PDF
*   Step 3: Extract Text from the PDF
*   Step 4: Count the Tokens in the Extracted Text
*   Step 5: Use LangChain to Generate a Summary of the Paper



### **Step 1: Import the Necessary Libraries**

In [ ]:
# !pip install PyPDF2
# !pip install tiktoken
# !pip install langchain_community

In [9]:
import os
import requests
from PyPDF2 import PdfReader
import tiktoken
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback

In [10]:
from secret_key import openapi_key
import os
os.environ['OPENAI_API_KEY'] = openapi_key
from langchain.llms import OpenAI

### **Step 2: Download and Read the PDF**

*   Define the path of the paper.
*   Read the PDF using **PdfReader**.
*   Print the number of pages.



In [12]:
# Define the path of the paper
PAPER_PATH = "arxiv_impact_of_GENAI.pdf"

# Read the PDF
reader = PdfReader(PAPER_PATH)

# Print the number of pages in the PDF
print(f"Number of pages: {len(reader.pages)}")

Number of pages: 9


### **Step 3: Extract Text from the PDF**

*   Initialize an empty list to store text parts.
*   Define a function to visit the body of the text.
*   Extract the text from each page of the PDF.
*   Join the parts of the text into a single string.
*   Print the extracted text.




In [ ]:
# There is also a library called as "pdfplumber" which is an OCR based extractor and is really useful when it comes to column text

In [13]:
# tm[a,b,c,d,e,f] --> Transformation Matrix

# a,b Conrols the text scaling and roatation
# c,d Control text skewing
# e,f Represents the x, y coordinate - text position

In [ ]:
# cm - Char Matrix
# Font Dict

In [25]:
# Initialize an empty list to store the parts of the text
parts = [] 

# Define a function to visit the body of the text
def visitor_body(text, cm, tm, fontDict, fontSize):
    y = tm[5]
    if y > 50 and y < 720:
        parts.append(text)

# Extract the text from each page of the PDF
for page in reader.pages:
    page.extract_text(visitor_text=visitor_body)

# Join the parts of the text into a single string
text_body = "".join(parts)

# Print the extracted text
print(text_body)


The Impact of Generative Artificial Intelligence
Kaichen Zhang1,2, Ohchan Kwon2, Hui Xiong1,2
1Hong Kong University of Science and Technology (Guangzhou)
2Hong Kong University of Science and Technology
kzhangbi@connect.ust.hk, ohchankw@ust.hk, xionghui@ust.hk
Abstract
The rise of generative artificial intelligence (AI) has sparked
concerns about its potential influence on unemployment and
market depression. This study addresses this concern by ex-
amining the impact of generative AI on product markets. To
overcome the challenge of causal inference, given the inher-
ent limitations of conducting controlled experiments, this pa-
per identifies an unanticipated and sudden leak of a highly
proficient image-generative AI as a novel instance of a ”natu-
ral experiment.” This AI leak spread rapidly, significantly re-
ducing the cost of generating anime-style images compared to
other styles, creating an opportunity for comparative assess-
ment. We collect real-world data from an artwork outsou

In [24]:
# Initialize an empty list to store the parts of the text
parts = [] 

# Define a function to visit the body of the text
def visitor_body(text, cm, tm, fontDict, fontSize):
    y = tm[5]
    if y > 50 and y < 720 and fontSize > 12:
        parts.append(text)

# Extract the text from each page of the PDF
for page in reader.pages:
    page.extract_text(visitor_text=visitor_body)

# Join the parts of the text into a single string
text_body = "".join(parts)

# Print the extracted text
print(text_body)


The Impact of Generative Artificial IntelligencearXiv:2311.07071v1  [econ.GN]  13 Nov 2023


### **Step 4: Count the Tokens in the Extracted Text**

*   Define a function to count the tokens in a text string.
*   Count the tokens in the extracted text.
*   Print the number of tokens.



In [15]:
# Define a function to count the tokens in a text string
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    num_tokens = len(encoding.encode(string))
    return num_tokens

# Count the tokens in the extracted text
num_tokens = num_tokens_from_string(text_body, "gpt-3.5-turbo")

# Print the number of tokens
print(num_tokens)

10556


### **Step 5: Use LangChain to Generate a Summary of the Paper**

*   Define the system and human prompts.
*   Create the **ChatPromptTemplate** object.
*   Create the **ChatOpenAI** object.
*   Create the **LLMChain** object.
*   Run the **LLMChain**.
*   Print the output.




In [16]:
# Define the system prompt
context_template = '''You are a helpful AI Researcher that specializes in analyzing ML, AI, and LLM papers.
Please use all your expertise to approach this task. Output your content in markdown format and include titles wherever relevant.'''

system_message_prompt = SystemMessagePromptTemplate.from_template(context_template)

In [17]:
# Define the human prompt
human_template = '''Please summarize this paper focusing on the key important takeaways for each section. 
Expand the summary on methods so they can be clearly understood. \n\n PAPER: \n\n{paper_content}'''

human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=human_template,
            input_variables=["paper_content"],
        )
    )


In [18]:
# Create the ChatPromptTemplate object
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [19]:
# Create the ChatOpenAI object
chat = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0.5)

In [20]:
# Create the LLMChain object
summary_chain = LLMChain(llm=chat, prompt=chat_prompt_template)

In [21]:
# Run the LLMChain and print the output
with get_openai_callback() as cb:
    output = summary_chain.run(text_body) # Text body has the extracted text from the pdf.
print(output)

# Summary of "The Impact of Generative Artificial Intelligence"

## Abstract
This paper examines the impact of generative artificial intelligence (AI) on product markets. The authors use an unanticipated leak of a highly proficient image-generative AI as a natural experiment to study the effects of generative AI on prices, order volume, and overall revenue. Surprisingly, the results show that generative AI lowers average prices but substantially boosts order volume and overall revenue. This counterintuitive finding suggests that generative AI confers benefits upon artists rather than detriments.

## Introduction
Generative AI has raised concerns about unemployment and market depression. Artists have protested against AI-generated images, and policymakers are considering regulations to limit the use of generative AI. However, there are differing views on the economic impact of generative AI. This paper aims to empirically examine the impact of generative AI on product markets to bridge 

### **Conclusion**

This demo provides a step-by-step guide on how to build an Arxiv paper summarizer using Python. By following these steps, you have created your own paper summarizer that can read a paper in one shot and generate a summary.

**Note:**
*   Save the output as a text file.
*   Name the text file **Summary.txt**.
*   This file will be used in the next session for benchmarking purposes.



In [26]:
with get_openai_callback() as cb:
    output = summary_chain.run(text_body)  # Runs the summarization
    
    
    print("\n Token Usage Details:")
    print(f"- Total Tokens Used: {cb.total_tokens}")
    print(f"- Prompt Tokens (input to GPT): {cb.prompt_tokens}")
    print(f"- Completion Tokens (output from GPT): {cb.completion_tokens}")
    print(f"- Estimated Cost: ${cb.total_cost:.5f}")


print("\n Summary Output:\n")
print(output)



 Token Usage Details:
- Total Tokens Used: 11186
- Prompt Tokens (input to GPT): 10642
- Completion Tokens (output from GPT): 544
- Estimated Cost: $0.03410

 Summary Output:

# Summary of "The Impact of Generative Artificial Intelligence"

## Abstract
This paper examines the impact of generative artificial intelligence (AI) on product markets. The study uses an unanticipated leak of a highly proficient image-generative AI as a natural experiment to overcome the challenge of causal inference. The results show that generative AI lowers average prices but substantially boosts order volume and overall revenue. This counterintuitive finding suggests that generative AI benefits artists rather than detriments them. The paper provides theoretical economic explanations for this unexpected phenomenon and highlights the potential of generative AI to foster market prosperity.

## Introduction
The introduction discusses the rise of generative AI and the concerns it has sparked regarding unemploym

In [27]:
# Lets save this output as summary

with open("SUMMARY.txt", "w", encoding="utf-8") as file:
    file.write(output)